In [2]:
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import re
import networkx as nx
import datetime as dt
import time

from cleaning_functions import *
from cleaners import *

%matplotlib notebook

In [111]:
df = pd.read_csv('cleaned_podcasts.csv', sep='\t', index_col=0)
df

,date,duration,episode,guests,hosts,podcast,title
0,2018-03-20 22:29:51,10603,1094,Brian Redban,Joe Rogan,The Joe Rogan Experience,#1094 - Brian Redban
1,2018-03-19 23:20:58,11990,1093,Owen Benjamin,Joe Rogan,The Joe Rogan Experience,#1093 - Owen Benjamin & Kurt Metzger
2,2018-03-19 23:20:58,11990,1093,Kurt Metzger,Joe Rogan,The Joe Rogan Experience,#1093 - Owen Benjamin & Kurt Metzger
3,2018-03-15 00:26:37,7725,1092,Mary Lynn Rajskub,Joe Rogan,The Joe Rogan Experience,#1092 - Mary Lynn Rajskub
4,2018-03-14 23:39:03,9406,1091,Daniele Bolelli,Joe Rogan,The Joe Rogan Experience,#1091 - Daniele Bolelli
5,2018-03-13 23:22:42,11795,1090,Andrew Santino,Joe Rogan,The Joe Rogan Experience,#1090 - Andrew Santino
6,2018-03-08 02:07:37,13856,1089,John Dudley,Joe Rogan,The Joe Rogan Experience,#1089 - John Dudley
7,2018-03-07 03:06:07,11805,1088,Bryan Callen,Joe Rogan,The Joe Rogan Experience,#1088 - Bryan Callen
8,2018-03-05 23:37:15,10793,1087,Sturgill Simpson,Joe Rogan,The Joe Rogan Experience,#1087 - Sturgill Simpson
9,2018-03-01 00:54:02,11608,1086,Rory Albanese,Joe Rogan,The Joe Rogan Experience,#1086 - Rory Albanese


In [112]:
split_hosts = splitDataFrameList(df, 'hosts', ', ')
split_hosts

G1 = nx.from_pandas_dataframe(split_hosts, 'guests', 'hosts', edge_attr=['date', 'duration', 'podcast'], create_using=nx.MultiDiGraph())

In [114]:

G1.edges(data=True)
#nx.average_shortest_path_length(G1)
#nx.eccentricity(G1,'Joe Rogan')
#nx.shortest_path_length(G1, "the hosts")
#nx.shortest_path(G1, 'President Barack Obama', 'Dan Harmon')
len(G1.edges())

34777

In [96]:
supersoul = xml_to_df('rss_files/supersoul.xml')

def clean_supersoul(supersoul):
        
    date_parser('\s\-0000$', supersoul)
    
    supersoul['guests'] = supersoul.title.str.extract(r'([^:]+): ',expand=True)
    
    guest_split(', Part', supersoul)
    guest_split(' Part', supersoul)
    
    guest_split_last('The ', supersoul)
    
    replace('India.Arie', 'India Arie', supersoul)
    replace('RuPaul Charles', 'RuPaul', supersoul)
    
    supersoul = supersoul[supersoul.guests.str.contains(r'Your ') == False]
    supersoul = supersoul[supersoul.guests.str.contains(r' the ') == False]
    
    supersoul = supersoul[pd.notnull(supersoul['guests'])]
    supersoul = splitDataFrameList(supersoul, 'guests', ', and ')
    supersoul = splitDataFrameList(supersoul, 'guests', ', & ')
    supersoul = splitDataFrameList(supersoul, 'guests', ', ')
    supersoul = splitDataFrameList(supersoul, 'guests', ',')
    supersoul = splitDataFrameList(supersoul, 'guests', ' &amp; ')
    supersoul = splitDataFrameList(supersoul, 'guests', ' & ')
    supersoul = splitDataFrameList(supersoul, 'guests', ' And ')
    supersoul = splitDataFrameList(supersoul, 'guests', ' and ')

    supersoul['guests'] = [g.rstrip() for g in supersoul['guests']]
    
    return supersoul

supersoul = clean_supersoul(supersoul)

supersoul

,date,duration,guests,title
0,2018-04-16 05:00:00,2366,Madonna Badger,Madonna Badger: Finding Light After Unimaginab...
1,2018-04-11 05:00:00,2249,Tim Storey,Tim Storey: How Do You Turn a Setback into a C...
2,2018-04-09 05:00:00,2275,Jack Kornfield,Jack Kornfield: Buddhism 101
3,2018-04-04 05:00:00,2038,Malala Yousafzai,Malala Yousafzai: What Is Your Defining Moment?
4,2018-04-02 05:00:00,2163,Carole Bayer Sager,Carole Bayer Sager: Music as Prayer
5,2018-03-28 05:00:00,2284,Rev. Ed Bacon,The Rev. Ed Bacon: Do You Recognize Grace?
6,2018-03-26 05:00:00,1762,Yara Shahidi,Yara Shahidi: Next-Generation Actress and Acti...
7,2018-03-21 05:00:00,1890,Lin-Manuel Miranda,Lin-Manuel Miranda: Creativity and Compassion
8,2018-03-19 05:00:00,1932,Trevor Noah,Trevor Noah: Born a Crime
9,2018-03-14 05:00:00,3444,Jimmy Kimmel,Jimmy Kimmel: What Do You Stand For?


In [90]:
dan_harris = xml_to_df('rss_files/dan_harris.xml')

def clean_dan_harris(dan_harris):
        
    date_parser('\sEDT$', dan_harris)
    
    dan_harris['guests'] = dan_harris.title.str.extract(r'\#\d+: ([^(]+)',expand=True)
    
    guest_split(', ', dan_harris)
    
    dan_harris = dan_harris[dan_harris.guests.str.contains(r'Your ') == False]
    
    dan_harris = dan_harris[pd.notnull(dan_harris['guests'])]
    dan_harris = splitDataFrameList(dan_harris, 'guests', ', and ')
    dan_harris = splitDataFrameList(dan_harris, 'guests', ', & ')
    dan_harris = splitDataFrameList(dan_harris, 'guests', ', ')
    dan_harris = splitDataFrameList(dan_harris, 'guests', ',')
    dan_harris = splitDataFrameList(dan_harris, 'guests', ' &amp; ')
    dan_harris = splitDataFrameList(dan_harris, 'guests', ' & ')
    dan_harris = splitDataFrameList(dan_harris, 'guests', ' And ')
    dan_harris = splitDataFrameList(dan_harris, 'guests', ' and ')

    dan_harris['guests'] = [g.rstrip() for g in dan_harris['guests']]
    
    return dan_harris

dan_harris = clean_dan_harris(dan_harris)

dan_harris

,date,duration,guests,title
0,2018-04-11 03:00:00,4495,Janice Marturano,"#130: Janice Marturano, How to Be a Better Boss"
1,2018-04-04 03:00:00,4047,Daniel Pink,"#129: Daniel Pink, 'When' Can Make a Big Diffe..."
2,2018-03-28 03:00:00,3438,Andrea Petersen,"#128: Andrea Petersen, Untangling from Anxiety"
3,2018-03-21 03:00:00,4117,Lewis Howes,"#127: Lewis Howes, Former Pro-Athlete on Remov..."
4,2018-03-14 03:00:00,2858,Hansa Bergwall,"#126: Hansa Bergwall, Reminding Us That We Die..."
5,2018-03-07 03:00:00,3027,Lt. Col. Jannell MacAulay,"#125: Lt. Col. Jannell MacAulay, Teaching 'Men..."
6,2018-02-28 03:00:00,4002,Rhonda Magee,"#124: Rhonda Magee, Law Professor Using Mindfu..."
7,2018-02-21 03:00:00,3542,Brad Katsuyama,"#123: Brad Katsuyama, Wall Street Reformer"
8,2018-02-14 03:00:00,3843,Bob Roth,"#122: Bob Roth, Meditation Teacher to the Stars"
9,2018-02-07 03:00:00,3338,Susan Kaiser Greenland,#121: Susan Kaiser Greenland and Annaka Harris...


In [83]:
ufc_unfiltered = xml_to_df('rss_files/ufc_unfiltered.xml')

def clean_ufc_unfiltered(ufc_unfiltered):
        
    date_parser('\s\-0000$', ufc_unfiltered)
    
    ufc_unfiltered['guests'] = ufc_unfiltered.title.str.extract(r'UF\d+: ([^(]+)',expand=True)
    
    guest_split(' Talks', ufc_unfiltered)
    guest_split(' Sounds', ufc_unfiltered)
    guest_split(' Joins', ufc_unfiltered)
    guest_split(' Is ', ufc_unfiltered)
    guest_split(' on ', ufc_unfiltered)
    guest_split(' Fights', ufc_unfiltered)
    guest_split(' Predicts', ufc_unfiltered)
    guest_split(' Finds', ufc_unfiltered)
    guest_split(' Discusses', ufc_unfiltered)
    guest_split(' Says', ufc_unfiltered)
    guest_split(' Taps', ufc_unfiltered)
    guest_split(' Sets', ufc_unfiltered)
    guest_split(' Feels', ufc_unfiltered)
    guest_split(' In-', ufc_unfiltered)
    guest_split(' Loves', ufc_unfiltered)
    
    guest_split_last('with ', ufc_unfiltered)
    guest_split_last('The Great ', ufc_unfiltered)
    
    replace('Roy Jones, Jr.', 'Roy Jones Jr.', ufc_unfiltered)
    
    ufc_unfiltered = ufc_unfiltered[pd.notnull(ufc_unfiltered['guests'])]
    ufc_unfiltered = splitDataFrameList(ufc_unfiltered, 'guests', ', and ')
    ufc_unfiltered = splitDataFrameList(ufc_unfiltered, 'guests', ', & ')
    ufc_unfiltered = splitDataFrameList(ufc_unfiltered, 'guests', ', ')
    ufc_unfiltered = splitDataFrameList(ufc_unfiltered, 'guests', ',')
    ufc_unfiltered = splitDataFrameList(ufc_unfiltered, 'guests', ' & ')
    ufc_unfiltered = splitDataFrameList(ufc_unfiltered, 'guests', ' And ')
    ufc_unfiltered = splitDataFrameList(ufc_unfiltered, 'guests', ' and ')

    ufc_unfiltered['guests'] = [g.rstrip() for g in ufc_unfiltered['guests']]
    
    ufc_unfiltered = ufc_unfiltered[ufc_unfiltered.guests.str.contains(r'UFC') == False]
    
    return ufc_unfiltered

ufc_unfiltered = clean_ufc_unfiltered(ufc_unfiltered)

ufc_unfiltered

,date,duration,guests,title
0,2018-04-17 07:01:00,5049,Daniel Cormier,"UF188: Daniel Cormier Sounds Off on Jon Jones,..."
2,2018-04-10 07:01:00,5054,Chuck Zito,"UF186: Recapping UFC 223, the Wildest Week in ..."
3,2018-04-05 07:01:00,4563,Dana White,UF185: Dana White Talks UFC 223 and Brian Orte...
4,2018-04-03 07:01:00,3944,Khabib Nurmagomedov,UF184: Khabib Nurmagomedov Joins In-Studio and...
5,2018-03-29 07:01:00,5363,Chris Gethard,UF183: Chris Gethard Taps Matt Serra (Kinda)
6,2018-03-27 07:01:00,3918,Mark Henry,UF182: The Great Mark Henry Talks Frankie Edga...
7,2018-03-22 01:40:00,3619,Paul Felder,UF181: Paul Felder Is Prepared for a Barnburne...
9,2018-03-15 02:33:36,4205,Rose Namajunas,UF179: Rose Namajunas Is on a Mission to Chang...
10,2018-03-13 05:30:00,4543,Amanda Nunes,UF178: Amanda Nunes Is Excited About the Women...
11,2018-03-08 08:01:00,4162,Joseph Benavidez,UF177: Joseph Benavidez on His Year Away from ...


In [78]:
chris_kresser = xml_to_df('rss_files/chris_kresser.xml')

def clean_chris_kresser(chris_kresser):
        
    date_parser('\s\+0000$', chris_kresser)
    
    chris_kresser['guests'] = chris_kresser.title.str.extract(r'[Ww]ith ([^(]+)',expand=True)
    
    guest_split_last('with ', chris_kresser)
    
    chris_kresser = chris_kresser[chris_kresser.guests.str.contains(r' the ') == False]
    
    chris_kresser = chris_kresser[pd.notnull(chris_kresser['guests'])]
    chris_kresser = splitDataFrameList(chris_kresser, 'guests', ', and ')
    chris_kresser = splitDataFrameList(chris_kresser, 'guests', ', & ')
    chris_kresser = splitDataFrameList(chris_kresser, 'guests', ', ')
    chris_kresser = splitDataFrameList(chris_kresser, 'guests', ',')
    chris_kresser = splitDataFrameList(chris_kresser, 'guests', ' & ')
    chris_kresser = splitDataFrameList(chris_kresser, 'guests', ' And ')
    chris_kresser = splitDataFrameList(chris_kresser, 'guests', ' and ')

    chris_kresser['guests'] = [g.rstrip() for g in chris_kresser['guests']]
    
    return chris_kresser

chris_kresser = clean_chris_kresser(chris_kresser)

chris_kresser

,date,duration,guests,title
0,2018-04-03 15:00:35,2859,Michael Ruscio,RHR: Practical Steps for Healing the Gut—with ...
1,2018-03-21 12:43:08,3537,Chris Masterjohn,RHR: How to Maximize Your Nutrient Intake—with...
2,2018-02-27 19:45:00,2700,Dr. Mark Hyman,RHR: What the Heck Should We Eat? With Dr. Mar...
3,2018-02-21 22:30:29,2962,Michelle Tam,"RHR: How to Make Paleo Cooking Fun, Easy and F..."
4,2018-01-16 21:47:00,2359,Sarah Ballantyne,RHR: Debunking Paleo Diet Myths—with Sarah Bal...
5,2018-01-09 18:15:53,3054,Steven Lin,RHR: How Dental Health Affects Your Whole Body...
6,2017-12-20 01:09:43,3520,Sal Di Stefano,RHR: The Importance of Strength Training—with ...
7,2017-12-06 14:58:22,2616,Melissa Hartwig,RHR: The Best Ways to Support Diet Change—with...
8,2017-04-06 15:00:40,3263,Sara Gottfried,RHR: How to Slow Down Aging—with Sara Gottfried
9,2017-03-23 20:48:29,3377,Robb Wolf,"RHR: Why We Gain Weight—Beyond Carbs, Fat & Pr..."


In [70]:
bulletproof_radio = xml_to_df('rss_files/bulletproof_radio.xml')

def clean_bulletproof_radio(bulletproof_radio):
        
    date_parser('\s\-0000$', bulletproof_radio)
    
    bulletproof_radio['guests'] = bulletproof_radio.title.str.extract(r'(?: [Ww]ith | - )([^(]+)',expand=True)
    
    guest_split('#', bulletproof_radio)
    guest_split(', Ph.D', bulletproof_radio)
    guest_split(', MD', bulletproof_radio)
    guest_split_last('," ', bulletproof_radio)
    guest_split_last('! ', bulletproof_radio)
    
    guest_split_last(' - ', bulletproof_radio)
    guest_split_last('with ', bulletproof_radio)
    
    guest_split(' -', bulletproof_radio)
    guest_split_last(' – ', bulletproof_radio)
    
    replace('Jayson & Mira Calton', 'Jayson Calton & Mira Calton', bulletproof_radio)
    replace('The Bare Minimum From NY Times Best-Selling Author  James Altucher ', 'James Altucher', bulletproof_radio)
    
    bulletproof_radio.replace('', np.nan, regex=True, inplace=True)
    bulletproof_radio.replace('', np.nan, regex=True, inplace=True)
    
    bulletproof_radio = bulletproof_radio[bulletproof_radio.guests.str.contains(r' Oil') == False]
    
    bulletproof_radio = bulletproof_radio[pd.notnull(bulletproof_radio['guests'])]
    bulletproof_radio = splitDataFrameList(bulletproof_radio, 'guests', ', and ')
    bulletproof_radio = splitDataFrameList(bulletproof_radio, 'guests', ', & ')
    bulletproof_radio = splitDataFrameList(bulletproof_radio, 'guests', ', ')
    bulletproof_radio = splitDataFrameList(bulletproof_radio, 'guests', ',')
    bulletproof_radio = splitDataFrameList(bulletproof_radio, 'guests', ' & ')
    bulletproof_radio = splitDataFrameList(bulletproof_radio, 'guests', ' And ')
    bulletproof_radio = splitDataFrameList(bulletproof_radio, 'guests', ' and ')

    bulletproof_radio['guests'] = [g.rstrip() for g in bulletproof_radio['guests']]
    bulletproof_radio['guests'] = [g.rstrip('.') for g in bulletproof_radio['guests']]

    bulletproof_radio.replace('', np.nan, regex=True, inplace=True)
    bulletproof_radio = bulletproof_radio[pd.notnull(bulletproof_radio['guests'])]
    
    return bulletproof_radio

bulletproof_radio = clean_bulletproof_radio(bulletproof_radio)

bulletproof_radio

,date,duration,guests,title
0,2018-04-17 16:20:57,4284,Dr. Dan Pompa,Your Toxic Family: Generational Toxicity - Dr....
1,2018-04-10 19:12:58,3336,Tyler LeBaron,Hacking with Hydrogen - Tyler LeBaron #482
2,2018-04-03 18:12:41,1010,Dr. Oz,America! Take Back Your Health - Dr. Oz #480
3,2018-03-29 18:24:13,3517,Jen Gotch,Fun with Depression! Jen Gotch #479
4,2018-03-27 17:37:40,4609,Dr. Thomas O’Bryan,The Autoimmune Fix - Dr. Thomas O’Bryan - #478
5,2018-03-22 17:47:56,4004,Naomi Whittel,Glow From The Inside Out: Autophagy and Women ...
6,2018-03-13 18:04:00,3262,Dawson Church,Tap into Tapping: The EFT Episode - Dawson Chu...
7,2018-03-08 16:37:08,2485,Dr. Kate Kass,Beyond Erectile Dysfunction - Dr. Kate Kass # 473
8,2018-03-06 20:19:49,3445,Dr Mark Atkinson,Be Unlimited - Dr Mark Atkinson # 472
9,2018-02-22 18:40:27,3260,Walter O'Brien,"IQ vs. EQ With ""Scorpion,"" Walter O'Brien #469"


In [47]:
james_altucher = xml_to_df('rss_files/james_altucher.xml')

def clean_james_altucher(james_altucher):
        
    date_parser('\s\+0000$', james_altucher)
    
    james_altucher['guests'] = james_altucher.title.str.extract(r'\d+ - ([^(]+): ',expand=True)
    
    guest_split(': ', james_altucher)
    guest_split(' - ', james_altucher)
    guest_split(' [', james_altucher)
    
    james_altucher = james_altucher[james_altucher.guests.str.contains(r'Anxiety') == False]
    
    james_altucher = james_altucher[pd.notnull(james_altucher['guests'])]
    james_altucher = splitDataFrameList(james_altucher, 'guests', ', and ')
    james_altucher = splitDataFrameList(james_altucher, 'guests', ', & ')
    james_altucher = splitDataFrameList(james_altucher, 'guests', ', ')
    james_altucher = splitDataFrameList(james_altucher, 'guests', ',')
    james_altucher = splitDataFrameList(james_altucher, 'guests', ' & ')
    james_altucher = splitDataFrameList(james_altucher, 'guests', ' And ')
    james_altucher = splitDataFrameList(james_altucher, 'guests', ' and ')

    james_altucher['guests'] = [g.rstrip() for g in james_altucher['guests']]
    
    return james_altucher

james_altucher = clean_james_altucher(james_altucher)

james_altucher

,date,duration,guests,title
0,2018-04-17 07:00:00,4590,Tony Rock,343 - Tony Rock: The Process to Get ANY Idea O...
1,2018-04-16 07:00:00,4106,Frank Oz,342 - Frank Oz: How to Create Something Origin...
2,2018-04-12 07:00:00,4025,Anders Ericsson,341 - Anders Ericsson [Anniversary Episode]: 7...
3,2018-04-10 07:00:00,2534,Don McLean,340 - Don McLean: Why You Should Follow Your I...
4,2018-04-09 07:00:00,3491,Tyra Banks,339 - Tyra Banks: How to Create New Opportunit...
5,2018-04-05 07:00:00,3597,Michio Kaku,338 - Michio Kaku: What The Future Holds… (How...
6,2018-04-03 07:00:00,4281,Robert Kurson,337 - Robert Kurson: The Leap of Faith Everyon...
7,2018-04-02 07:00:00,4070,Steve Case,336 - Steve Case [Anniversary Episode]: An Ent...
8,2018-03-29 07:00:00,2551,Wynton Marsalis,335 - Wynton Marsalis: How to Find the Jazz of...
9,2018-03-27 07:00:00,4895,Brad Meltzer,334 - Brad Meltzer: Creating a Bestselling Nov...


In [40]:
politically_reactive = xml_to_df('rss_files/politically_reactive.xml')

def clean_politically_reactive(politically_reactive):
        
    date_parser('\s\-0000$', politically_reactive)
    
    politically_reactive['guests1'] = politically_reactive.title.str.extract(r'([\w\s\.\,\'\`\’\&\"\-\$]+) on ',expand=True)
    politically_reactive['guests2'] = politically_reactive.title.str.extract(r' with ([\w\s\.\,\'\`\’\&\"\-\$]+)',expand=True)

    politically_reactive.replace(np.nan, '', regex=True, inplace=True)
    politically_reactive['guests'] = politically_reactive['guests1'] + politically_reactive['guests2']  
    politically_reactive = politically_reactive.drop('guests1', 1)
    politically_reactive = politically_reactive.drop('guests2', 1)
    politically_reactive.replace('', np.nan, regex=True, inplace=True)
    
    guest_split(' of ', politically_reactive)

    guest_split_last('Journalist ', politically_reactive)
    guest_split_last("YouTube's ", politically_reactive)
    guest_split_last("CNN’s ", politically_reactive)
    guest_split_last('Comedian ', politically_reactive)
    
    for index, row in politically_reactive.iterrows():
        if(pd.notnull(row['guests'])):
            politically_reactive.at[index, 'guests'] = row['guests'].title()
    
    politically_reactive = politically_reactive[politically_reactive.guests.str.contains(r'Truth ') == False]
    
    politically_reactive = politically_reactive[pd.notnull(politically_reactive['guests'])]
    politically_reactive = splitDataFrameList(politically_reactive, 'guests', ', and ')
    politically_reactive = splitDataFrameList(politically_reactive, 'guests', ', & ')
    politically_reactive = splitDataFrameList(politically_reactive, 'guests', ', ')
    politically_reactive = splitDataFrameList(politically_reactive, 'guests', ',')
    politically_reactive = splitDataFrameList(politically_reactive, 'guests', ' & ')
    politically_reactive = splitDataFrameList(politically_reactive, 'guests', ' And ')
    politically_reactive = splitDataFrameList(politically_reactive, 'guests', ' and ')

    politically_reactive['guests'] = [g.rstrip() for g in politically_reactive['guests']]
    
    politically_reactive = politically_reactive[politically_reactive.guests.str.contains(r'Talking ') == False]
    politically_reactive = politically_reactive[politically_reactive.guests.str.contains(r' The ') == False]
    
    return politically_reactive

politically_reactive = clean_politically_reactive(politically_reactive)

politically_reactive

,date,duration,guests,title
0,2018-04-02 21:00:00,81,Mehdi Hasan,Introducing Deconstructed with Mehdi Hasan
1,2017-09-14 09:52:27,4835,Bell Hooks,Speaking with bell hooks & Talking DACA
3,2017-09-07 07:00:00,4189,Dr. Keeanga-Yamahtta Taylor,"Disaster Capitalism and How To Fight It, with ..."
4,2017-09-07 07:00:00,4189,Mustafa Santiago Ali,"Disaster Capitalism and How To Fight It, with ..."
6,2017-07-27 10:30:00,3963,Killer Mike,"Killer Mike of Run the Jewels on Allyship, Equ..."
7,2017-06-22 07:00:00,3080,Janet Mock,Janet Mock on Surpassing Certainty
8,2017-06-01 07:00:00,3617,Jill Filipovic,Jill Filipovic on the Feminist Pursuit of Happ...
9,2017-05-18 07:01:42,3479,Kate Schatz,Kate Schatz and Miriam Klein Stahl on Rad Amer...
10,2017-05-18 07:01:42,3479,Miriam Klein Stahl,Kate Schatz and Miriam Klein Stahl on Rad Amer...
11,2017-05-04 07:00:00,3144,Jeremy Scahill,Getting Adversarial with Journalist Jeremy Sca...


In [30]:
friends_like_these = xml_to_df('rss_files/friends_like_these.xml')

def clean_friends_like_these(friends_like_these):
        
    date_parser('\s\-0000$', friends_like_these)
    
    friends_like_these['guests'] = friends_like_these['title']  #.title.str.extract(r'(?:\"[^\"]+\"\s)(?:w\/ |with )([^(]+)',expand=True)
    
    guest_split_last('with ', friends_like_these)
    guest_split_last('w/ ', friends_like_these)
    
    guest_split('"', friends_like_these)
    guest_split('“', friends_like_these)
    
    friends_like_these.replace('', np.nan, inplace=True)
    
    friends_like_these = friends_like_these[friends_like_these.guests.str.contains(r'Episode') == False]
    friends_like_these = friends_like_these[friends_like_these.guests.str.contains(r'EPISODE') == False]
    friends_like_these = friends_like_these[friends_like_these.guests.str.contains(r'Special') == False]
    friends_like_these = friends_like_these[friends_like_these.guests.str.contains(r'pitchforks') == False]
    friends_like_these = friends_like_these[friends_like_these.guests.str.contains(r' a ') == False]
    
    friends_like_these = friends_like_these[pd.notnull(friends_like_these['guests'])]
    friends_like_these = splitDataFrameList(friends_like_these, 'guests', ', and ')
    friends_like_these = splitDataFrameList(friends_like_these, 'guests', ', & ')
    friends_like_these = splitDataFrameList(friends_like_these, 'guests', ', ')
    friends_like_these = splitDataFrameList(friends_like_these, 'guests', ',')
    friends_like_these = splitDataFrameList(friends_like_these, 'guests', ' & ')
    friends_like_these = splitDataFrameList(friends_like_these, 'guests', ' And ')
    friends_like_these = splitDataFrameList(friends_like_these, 'guests', ' and ')

    friends_like_these['guests'] = [g.rstrip() for g in friends_like_these['guests']]
    
    return friends_like_these

friends_like_these = clean_friends_like_these(friends_like_these)

friends_like_these

,date,duration,guests,title
0,2018-04-13 07:01:00,3590,Keah Brown,"""Talking Ableism, Hashtag Appropriation and Ro..."
1,2018-03-30 07:01:00,2472,Dave Weigel,“They’re standing at the podium” w/ Dave Weigel
2,2018-03-09 08:01:00,3690,Christian Picciolini,“The Average American White Racist” with Chris...
3,2018-03-02 08:01:00,2864,Jamil Smith,“Black Future Month” with Jamil Smith
4,2018-02-23 08:01:00,4123,McKay Coppins,“Mormon Pundit Mode” with McKay Coppins
5,2018-02-09 08:01:00,4445,Melissa Febos,Not Your Mama's Interracial Relationship with ...
6,2018-02-09 08:01:00,4445,Quinta Brunson,Not Your Mama's Interracial Relationship with ...
7,2018-02-02 08:01:00,2907,Rick Wilson,“A Party of Men and Mobs” with Rick Wilson
8,2018-01-26 08:01:00,4867,Katie Notopoulos,"“If You Can’t Vlog It, It Didn’t Happen” with ..."
9,2018-01-26 08:01:00,4867,Jemar Tisby,"“If You Can’t Vlog It, It Didn’t Happen” with ..."


In [13]:
axe_files = xml_to_df('rss_files/axe_files.xml')

def clean_axe_files(axe_files):
        
    date_parser('\s\-0000$', axe_files)
    
    axe_files['guests'] = axe_files.title.str.extract(r'Ep. \d+ - ([^(]+)',expand=True)
    
    axe_files = axe_files[pd.notnull(axe_files['guests'])]

    axe_files['guests'] = [g.rstrip() for g in axe_files['guests']]
    
    return axe_files

axe_files = clean_axe_files(axe_files)

axe_files

,title,date,duration,guests
0,Ep. 234 - Zalmay Khalilzad,2018-04-16 11:57:24,3848,Zalmay Khalilzad
1,Ep. 233 - Ed Miliband,2018-04-12 09:00:00,4050,Ed Miliband
2,Ep. 232 - Stephanie Schriock,2018-04-09 09:00:00,3722,Stephanie Schriock
3,Ep. 231 - Major Garrett,2018-04-05 09:00:00,3925,Major Garrett
4,Ep. 230 - Charles Barkley,2018-04-01 01:00:00,3818,Charles Barkley
5,Ep. 229 - Andrea Mitchell,2018-03-29 09:00:00,3825,Andrea Mitchell
6,Ep. 228 - Michael Phelps,2018-03-26 20:11:23,3583,Michael Phelps
7,Ep. 227 - Chris Cuomo,2018-03-22 08:00:00,3738,Chris Cuomo
8,Ep. 226 - Peter Wehner,2018-03-19 09:00:00,3090,Peter Wehner
9,Ep. 225 - Ana Marie Cox,2018-03-15 10:00:00,3804,Ana Marie Cox


In [8]:
this_life = xml_to_df('rss_files/this_life.xml')

def clean_this_life(this_life):
        
    date_parser('\s\+0000$', this_life)
    
    this_life['guests'] = this_life.title.str.extract(r'\d+(?:\s?: | - )([^(]+)',expand=True)
    
    guest_split(' Returns', this_life)
    guest_split(' VS. PTSD', this_life)
    guest_split(' PTSD', this_life)
    guest_split(' Talks', this_life)
    
    guest_split_last(' - ', this_life)
    guest_split_last('With ', this_life)
    guest_split_last('Soberoctobert ', this_life)
    guest_split_last('Interventionist ', this_life)
    
    this_life = this_life[this_life.guests.str.contains(r'This ') == False]
    this_life = this_life[this_life.guests.str.contains(r'Facing ') == False]
    this_life = this_life[this_life.guests.str.contains(r'At The ') == False]
    
    this_life = this_life[pd.notnull(this_life['guests'])]
    this_life = splitDataFrameList(this_life, 'guests', ', and ')
    this_life = splitDataFrameList(this_life, 'guests', ', & ')
    this_life = splitDataFrameList(this_life, 'guests', ', ')
    this_life = splitDataFrameList(this_life, 'guests', ',')
    this_life = splitDataFrameList(this_life, 'guests', ' & ')
    this_life = splitDataFrameList(this_life, 'guests', ' And ')
    this_life = splitDataFrameList(this_life, 'guests', ' and ')
    this_life = splitDataFrameList(this_life, 'guests', ' / ')
    this_life = splitDataFrameList(this_life, 'guests', ' /')
    this_life = splitDataFrameList(this_life, 'guests', ' Meets ')

    this_life['guests'] = [g.rstrip() for g in this_life['guests']]
    
    replace('Emily', 'Emily Morse', this_life)
    replace('Christina P', 'Christina Pazsitzky', this_life)
    
    return this_life

this_life = clean_this_life(this_life)

this_life

,date,duration,guests,title
0,2018-04-16 16:59:11,3705,Claudia Christian,#YOULIVE 129 - Claudia Christian and Dr John M...
1,2018-04-16 16:59:11,3705,Dr John Mendelson,#YOULIVE 129 - Claudia Christian and Dr John M...
2,2018-04-10 00:17:26,4150,Joe Lockett,#YouLive 128 - Joe Lockett And Michael Lohan
3,2018-04-10 00:17:26,4150,Michael Lohan,#YouLive 128 - Joe Lockett And Michael Lohan
4,2018-04-02 17:38:18,4410,PJ Schrantz,#YOULIVE 127 - PJ Schrantz & Dopey Guys
5,2018-04-02 17:38:18,4410,Dopey Guys,#YOULIVE 127 - PJ Schrantz & Dopey Guys
6,2018-03-26 18:02:57,3214,Carla Spalding,#YOULIVE 126 - Carla Spalding
7,2018-03-19 20:02:05,3804,Mary Carey,#YOULIVE 125 - Mary Carey
8,2018-03-12 19:51:30,5353,Todd Zalkins,#YOULIVE 124 - Interventionist Todd Zalkins
9,2018-03-05 15:42:46,3569,Dr. Lisa Strohman,#YOULIVE 123 - Dr. Lisa Strohman And Gene Rugala


In [468]:
vlad_couch = xml_to_df('rss_files/vlad_couch.xml')

def clean_vlad_couch(vlad_couch):
        
    date_parser('\s\+0000$', vlad_couch)
    
    vlad_couch['guests'] = vlad_couch.title.str.extract(r'([^(]+)',expand=True)
    
    guest_split(' of ', vlad_couch)
    guest_split(' Talks ', vlad_couch)
    
    vlad_couch = vlad_couch[vlad_couch.guests.str.contains(r' Details ') == False]
    
    vlad_couch = vlad_couch[pd.notnull(vlad_couch['guests'])]
    vlad_couch = splitDataFrameList(vlad_couch, 'guests', ', and ')
    vlad_couch = splitDataFrameList(vlad_couch, 'guests', ', & ')
    vlad_couch = splitDataFrameList(vlad_couch, 'guests', ', ')
    vlad_couch = splitDataFrameList(vlad_couch, 'guests', ',')
    vlad_couch = splitDataFrameList(vlad_couch, 'guests', ' & ')
    vlad_couch = splitDataFrameList(vlad_couch, 'guests', ' And ')
    vlad_couch = splitDataFrameList(vlad_couch, 'guests', ' and ')

    vlad_couch['guests'] = [g.rstrip() for g in vlad_couch['guests']]
    
    replace('Charlamagne', 'Charlamagne Tha God', vlad_couch)
    
    return vlad_couch

vlad_couch = clean_vlad_couch(vlad_couch)

vlad_couch

,date,duration,guests,title
0,2017-10-10 21:06:58,7694,D.L. Hughley,D.L. Hughley (Episode 129)
1,2017-10-03 20:23:20,4554,Montana,Montana of 300 (Episode 128)
2,2017-09-29 21:23:55,5234,Too Short,Too Short (Episode 127)
3,2017-08-22 18:49:01,5588,Lord Jamar,Lord Jamar (Episode 126)
4,2017-08-18 20:39:11,1479,Splinta,Splinta (Episode 125)
5,2017-08-17 19:26:41,2496,Dae Dae,Dae Dae (Episode 124)
6,2017-08-16 18:50:20,2585,Sad Boy Loko,Sad Boy Loko (Episode 123)
7,2017-08-11 15:10:55,5177,Havoc,Havoc of Mobb Deep (Episode 122)
8,2017-08-10 19:24:46,5776,David Banner,David Banner (Episode 121)
9,2017-08-03 17:58:54,3892,Damian Marley,Damian Marley (Episode 120)


In [451]:
rap_radar = xml_to_df('rss_files/rap_radar.xml')

def clean_rap_radar(rap_radar):
        
    date_parser('\s\+0000$', rap_radar)
    
    rap_radar['guests'] = rap_radar.title.str.extract(r'E[Pp] \d+: ([\w\s\.\,\'\`\’\&\"\-\$]+)',expand=True)
    
    replace('The Breakfast Club', 'DJ Envy, Angela Yee, Charlamagne Tha God', rap_radar)
    
    rap_radar = rap_radar[pd.notnull(rap_radar['guests'])]
    rap_radar = splitDataFrameList(rap_radar, 'guests', ', and ')
    rap_radar = splitDataFrameList(rap_radar, 'guests', ', & ')
    rap_radar = splitDataFrameList(rap_radar, 'guests', ', ')
    rap_radar = splitDataFrameList(rap_radar, 'guests', ',')
    rap_radar = splitDataFrameList(rap_radar, 'guests', ' & ')
    rap_radar = splitDataFrameList(rap_radar, 'guests', ' And ')
    rap_radar = splitDataFrameList(rap_radar, 'guests', ' and ')

    rap_radar['guests'] = [g.rstrip() for g in rap_radar['guests']]
    
    return rap_radar

rap_radar = clean_rap_radar(rap_radar)

rap_radar

,date,duration,guests,title
0,2017-04-28 11:00:11,2949,Tee Grizzley,EP 100: Tee Grizzley
1,2017-04-21 11:00:03,4331,Kid Capri,EP 99: Kid Capri
2,2017-03-24 11:00:45,4685,Bink,EP 98: Bink
3,2017-03-03 12:00:31,3692,Run The Jewels,EP 97: Run The Jewels
4,2017-02-24 21:16:31,3873,PnB Rock,EP 96: PnB Rock
5,2017-02-06 12:00:23,5961,Stephen Hill,EP 95: Stephen Hill
6,2017-01-24 12:00:16,3107,Nick Grant,EP 94: Nick Grant
7,2017-01-23 12:00:54,5028,Troy Carter,EP 93: Troy Carter
8,2017-01-20 12:00:34,3207,6LACK,EP 92: 6LACK
9,2017-01-19 12:00:25,3647,BJ The Chicago Kid,EP 91: BJ The Chicago Kid


In [430]:
snoop = xml_to_df('rss_files/snoop.xml')

def clean_snoop(snoop):
        
    date_parser('\s\+0000$', snoop)
    
    snoop['guests'] = snoop.title.str.extract(r'Ep. \d+ - ([\w\s\.\,\'\`\’\&\"\-\$]+)',expand=True)
    
    snoop = snoop[pd.notnull(snoop['guests'])]
    snoop = splitDataFrameList(snoop, 'guests', ', and ')
    snoop = splitDataFrameList(snoop, 'guests', ', & ')
    snoop = splitDataFrameList(snoop, 'guests', ', ')
    snoop = splitDataFrameList(snoop, 'guests', ',')
    snoop = splitDataFrameList(snoop, 'guests', ' & ')
    snoop = splitDataFrameList(snoop, 'guests', ' And ')
    snoop = splitDataFrameList(snoop, 'guests', ' and ')
    
    snoop = snoop[snoop.guests.str.contains(r'Special') == False]

    snoop['guests'] = [g.rstrip() for g in snoop['guests']]
    
    return snoop

snoop = clean_snoop(snoop)

snoop

,date,duration,guests,title
0,2018-04-04 21:39:40,1778,K Camp,GGN Podcast Ep. 106 - K Camp
1,2018-03-22 01:46:17,1771,Martin Lawrence,GGN Podcast Ep. 105 - Martin Lawrence
2,2018-03-08 01:40:35,1540,G Perico,GGN Podcast Ep. 104 - G Perico
3,2018-02-20 20:16:47,3949,Ernest Dickerson,GGN Podcast Ep. 103 - Ernest Dickerson
4,2018-02-07 01:21:11,1847,Lena Waithe,GGN Podcast Ep. 102 - Lena Waithe
5,2018-01-19 11:34:55,2598,Kathy Bates,GGN Podcast Ep. 101 - Kathy Bates
6,2017-12-22 06:29:28,1696,Christmas Special 2017,GGN Podcast Ep. 100 - Christmas Special 2017
7,2017-12-19 05:20:30,2813,RJ,GGN Podcast Ep. 99 - RJ
8,2017-12-05 06:05:06,1628,Chief Keef,GGN Podcast Ep. 98 - Chief Keef
9,2017-11-21 01:10:53,1616,Desiigner,GGN Podcast Ep. 97 - Desiigner


In [445]:
andy_cohen = xml_to_df('rss_files/andy_cohen.xml')

def clean_andy_cohen(andy_cohen):
        
    date_parser('\s\-0000$', andy_cohen)
    
    andy_cohen['guests'] = andy_cohen.title.str.extract(r'(?:[\d\/] [-–] )?([^\d^\/]+)(?: [-–] [\d\/])?',expand=True)

    guest_split(' (', andy_cohen)
    guest_split(' - ', andy_cohen)
    
    andy_cohen = andy_cohen[pd.notnull(andy_cohen['guests'])]
    andy_cohen = splitDataFrameList(andy_cohen, 'guests', ', and ')
    andy_cohen = splitDataFrameList(andy_cohen, 'guests', ', & ')
    andy_cohen = splitDataFrameList(andy_cohen, 'guests', ', ')
    andy_cohen = splitDataFrameList(andy_cohen, 'guests', ',')
    andy_cohen = splitDataFrameList(andy_cohen, 'guests', ' & ')
    andy_cohen = splitDataFrameList(andy_cohen, 'guests', ' And ')
    andy_cohen = splitDataFrameList(andy_cohen, 'guests', ' and ')
    
    andy_cohen = andy_cohen[andy_cohen.guests.str.contains(r'the N') == False]
    andy_cohen = andy_cohen[andy_cohen.guests.str.contains(r'Special') == False]
    andy_cohen = andy_cohen[andy_cohen.guests.str.contains(r' Live') == False]
    andy_cohen = andy_cohen[andy_cohen.guests.str.contains(r' Cast') == False]
    
    andy_cohen['guests'] = [g.rstrip() for g in andy_cohen['guests']]
    
    return andy_cohen

andy_cohen = clean_andy_cohen(andy_cohen)

andy_cohen

,date,duration,guests,title
0,2018-04-13 20:00:00,1419,Snoop Dogg,Snoop Dogg & Anna Faris - 04/12/18
1,2018-04-13 20:00:00,1419,Anna Faris,Snoop Dogg & Anna Faris - 04/12/18
2,2018-04-12 16:00:12,1914,Dorinda Medley,Dorinda Medley and the Night of 31 Doorbells -...
5,2018-04-10 20:00:00,1617,Ricky Martin,Ricky Martin & Milo Ventimiglia - 04/09/18
6,2018-04-10 20:00:00,1617,Milo Ventimiglia,Ricky Martin & Milo Ventimiglia - 04/09/18
7,2018-04-09 20:00:00,1692,NeNe Leakes,NeNe Leakes & Lena Waithe - 04/08/18
8,2018-04-09 20:00:00,1692,Lena Waithe,NeNe Leakes & Lena Waithe - 04/08/18
9,2018-04-06 20:00:00,1525,Keri Russell,Keri Russell & Matthew Rhys - 04/05/18
10,2018-04-06 20:00:00,1525,Matthew Rhys,Keri Russell & Matthew Rhys - 04/05/18
11,2018-04-05 20:00:00,1623,Luann de Lesseps,Luann de Lesseps & Jackie Hoffman - 04/04/18


In [424]:
bert_kreischer = xml_to_df('rss_files/bert_kreischer.xml')

def clean_bert_kreischer(bert_kreischer):
    
    bert_kreischer = bert_kreischer.loc[bert_kreischer.title.str.contains('Call In Sick To Work Show') == False]

    ## Set Episode numbers
    
    bert_kreischer['episode'] = bert_kreischer.title.str.extract(r'\#\s?([\d.]+)',expand=True)

    date_parser('\s\+0000$', bert_kreischer)
    
    ## Set guest list
    
    for index, row in bert_kreischer.iterrows():
        if(pd.notnull(row['title'])):
            bert_kreischer.at[index, 'title'] = row['title'].title()
    
    bert_kreischer['guests'] = bert_kreischer.title.str.extract(r'\#[\s-]?[\d\.-]+\s(?:[-\s]+)?(?:\([\w\s]+\))?([\w\s\.\,&/\'\-"]+)',expand=True)
    
    for index, row in bert_kreischer.iterrows():
        if(pd.isnull(row['guests'])):
            bert_kreischer.at[index, 'guests'] = row['title']
    
    guests_no_me =[]
    for g in bert_kreischer['guests']:
        #guests_no_me.append(re.sub('[\s&|,\sand] ME', '', str(g), flags=re.IGNORECASE))
        guests_no_me.append(re.sub('(, and ME|, & ME| & ME|...AND ME|ME, & )', '', str(g), flags=re.IGNORECASE))
        
    bert_kreischer['guests'] = guests_no_me
    
    for index, row in bert_kreischer.iterrows():
        if(row['title']=='Jon Heffron, Jon Moore, & Me'):
            bert_kreischer.at[index, 'episode'] = '4'
            bert_kreischer.at[index, 'guests'] = 'Jon Heffron, Jon Moore'
        if(row['episode']=='1'):  
            bert_kreischer.at[index, 'guests'] = 'Joey Diaz, Tom Segura, My Dad'

    bert_kreischer['guests'] = [g.rstrip() for g in bert_kreischer['guests']]
    bert_kreischer['guests'] = [g.rstrip('.') for g in bert_kreischer['guests']]
    bert_kreischer['guests'] = [g.lstrip() for g in bert_kreischer['guests']]
            
    guest_split_last('W/ ', bert_kreischer)
    guest_split_last("'S ", bert_kreischer)
    guest_split_last("Guest ", bert_kreischer)
    guest_split_last("The Complete ", bert_kreischer)
    guest_split_last("Creator ", bert_kreischer)
    guest_split_last("By ", bert_kreischer)
            
    guest_split(' In', bert_kreischer)
    guest_split(' - ', bert_kreischer)
            
    replace('The Jeffs', 'Jeff Johnson & Jeff Hinman', bert_kreischer)
    replace('Redban', 'Brian Redban', bert_kreischer)
    replace('Push', 'Christina Pazsitzky', bert_kreischer)
    replace('The Chive', 'Mac Faulkner', bert_kreischer)
    replace('Sam', 'Sam Roberts', bert_kreischer)
    replace('Fitness Special W', 'Chris Tye Walker', bert_kreischer)
    replace('Sunday Special... A Chiropractor, His Hot Chic, John Bush', 'John Bush', bert_kreischer)
    replace('Mick Foley W', 'Mick Foley, Sam Roberts', bert_kreischer)
    replace('Kevin Heffernan, Steve LemMe', 'Kevin Heffernan, Steve Lemme', bert_kreischer)
    #replace('Just Me', np.nan, bert_kreischer)
    replace('Josh Wolf The Last 10 Minutes', 'Josh Wolf', bert_kreischer)
    
    guest_split(' From', bert_kreischer)
    
    replace('The Rolling Stone Magazine Article "The Undergraduate"', np.nan, bert_kreischer)
    replace('The End Of The Undergraduate', np.nan, bert_kreischer)
    replace('#146 – Book Club Crash – So You’ve Been Publicly Shamed, by Jon Ronson', 'Jon Ronson', bert_kreischer)
    #replace('Co', 'Jonathan Steinberg', bert_kreischer)
    replace('Joey "Coco" Diaz', 'Joey Diaz', bert_kreischer)
    replace('Doug Loves Bert', 'Doug Benson', bert_kreischer)
    replace('Annie Lederman& Me', 'Annie Lederman', bert_kreischer)
    replace('Live', 'Doug Stanhope', bert_kreischer)
    replace('Danish & O', 'Danish & O’Neill', bert_kreischer)
    replace('Ari Shaffir ', 'Ari Shaffir', bert_kreischer)

    bert_kreischer = bert_kreischer[bert_kreischer.guests.str.contains('Daddy ') == False]
    bert_kreischer = bert_kreischer[bert_kreischer.guests.str.contains('Best Of ') == False]
    bert_kreischer = bert_kreischer[bert_kreischer.guests.str.contains('Special') == False]
    
    ## Split Guests
    
    bert_kreischer = bert_kreischer[pd.notnull(bert_kreischer['guests'])]
    bert_kreischer = splitDataFrameList(bert_kreischer, 'guests', ', and ')
    bert_kreischer = splitDataFrameList(bert_kreischer, 'guests', ', And ')
    bert_kreischer = splitDataFrameList(bert_kreischer, 'guests', ', & ')
    bert_kreischer = splitDataFrameList(bert_kreischer, 'guests', ', ')
    bert_kreischer = splitDataFrameList(bert_kreischer, 'guests', ',')
    bert_kreischer = splitDataFrameList(bert_kreischer, 'guests', ' & ')
    bert_kreischer = splitDataFrameList(bert_kreischer, 'guests', ' And ')
    bert_kreischer = splitDataFrameList(bert_kreischer, 'guests', ' and ')
    
    replace('Wheeler Walker Jr', 'Wheeler Walker Jr.', bert_kreischer)
    replace('Joey Coco Diaz', 'Joey Diaz', bert_kreischer)
    replace('Sam "Primetime" Roberts', 'Sam Roberts', bert_kreischer)
    replace('Me', np.nan, bert_kreischer)
    replace('Just Me', np.nan, bert_kreischer)
    replace('Redban', 'Brian Redban', bert_kreischer)
    replace('A Little Leeann', 'Leeann Kreischer', bert_kreischer)
    replace('Leeann', 'Leeann Kreischer', bert_kreischer)
    replace('Danish', 'Jeff Danis', bert_kreischer)
    replace('O’Neill', 'Ryan O’Neill', bert_kreischer)
    replace("O'Neill", 'Ryan O’Neill', bert_kreischer)
    replace("O'Neil", 'Ryan O’Neill', bert_kreischer)
    
    bert_kreischer = bert_kreischer[bert_kreischer['guests'].str.len() < 22]
    bert_kreischer = bert_kreischer[bert_kreischer.guests.str.contains(r'The ') == False]

    bert_kreischer.replace('', np.nan, inplace=True)
    bert_kreischer = bert_kreischer[pd.notnull(bert_kreischer['guests'])]
    
    return bert_kreischer

bert_kreischer = clean_bert_kreischer(bert_kreischer)
bert_kreischer[100:150]

,date,duration,episode,guests,title
100,2016-11-02 04:48:51,6085,205,Jim Norton,"#205 - Jim Norton, Sam Roberts, & Me"
101,2016-11-02 04:48:51,6085,205,Sam Roberts,"#205 - Jim Norton, Sam Roberts, & Me"
102,2016-10-26 07:31:32,8929,204,Kurt Metzger,#204 - Kurt Metzger & Me
103,2016-10-19 04:28:59,8158,203,Mike Young,#203 - Mike Young & Me
104,2016-10-12 04:27:58,11120,202,Jeff Danis,#202 - Danish & O'Neil & Me
105,2016-10-12 04:27:58,11120,202,Ryan O’Neill,#202 - Danish & O'Neil & Me
106,2016-10-10 19:04:31,4899,201,Zane Lamprey,#201 - Zane Lamprey & Me
107,2016-10-05 04:05:59,9065,200,Bill Burr,"#200 - Bill Burr, Tom Segura, & Me"
108,2016-10-05 04:05:59,9065,200,Tom Segura,"#200 - Bill Burr, Tom Segura, & Me"
109,2016-09-28 07:04:11,9859,199,Moonshine,"#199 - Moonshine, Trae Crowder, Corey Forreste..."


In [124]:
cam_rhea = xml_to_df('rss_files/cam_rhea.xml')

def clean_cam_rhea(cam_rhea):
        
    date_parser('\s\-0000$', cam_rhea)
    
    cam_rhea['guests'] = cam_rhea.title.str.extract(r'([\w\s\.\,\'\`\’\&\"\-]+)',expand=True)
    
    guest_split(' and host', cam_rhea)
    
    cam_rhea = cam_rhea[pd.notnull(cam_rhea['guests'])]
    cam_rhea = splitDataFrameList(cam_rhea, 'guests', ', and ')
    cam_rhea = splitDataFrameList(cam_rhea, 'guests', ', & ')
    cam_rhea = splitDataFrameList(cam_rhea, 'guests', ', ')
    cam_rhea = splitDataFrameList(cam_rhea, 'guests', ',')
    cam_rhea = splitDataFrameList(cam_rhea, 'guests', ' & ')
    cam_rhea = splitDataFrameList(cam_rhea, 'guests', ' And ')
    cam_rhea = splitDataFrameList(cam_rhea, 'guests', ' and ')

    replace('Paul F. Tompkins as "Cake Boss," Rory Scovel', 'Paul F. Tompkins, Rory Scovel', cam_rhea)
    replace('The Sklar Brothers', 'Jason Sklar, Randy Sklar', cam_rhea)
    replace('The Walsh Brothers', 'Dave Walsh, Chris Walsh', cam_rhea)
    cam_rhea = splitDataFrameList(cam_rhea, 'guests', ', ')

    cam_rhea['guests'] = [g.rstrip() for g in cam_rhea['guests']]
    
    return cam_rhea

cam_rhea = clean_cam_rhea(cam_rhea)

cam_rhea

,date,duration,guests,title
0,2018-04-12 04:00:00,4271,Guy Branum,"Guy Branum, Darcy Michael, Kyle Ayers and host..."
1,2018-04-12 04:00:00,4271,Darcy Michael,"Guy Branum, Darcy Michael, Kyle Ayers and host..."
2,2018-04-12 04:00:00,4271,Kyle Ayers,"Guy Branum, Darcy Michael, Kyle Ayers and host..."
3,2018-04-05 04:00:00,4863,Jonah Ray,"Jonah Ray, Megan Gailey, Joyelle Johnson, Zach..."
4,2018-04-05 04:00:00,4863,Megan Gailey,"Jonah Ray, Megan Gailey, Joyelle Johnson, Zach..."
5,2018-04-05 04:00:00,4863,Joyelle Johnson,"Jonah Ray, Megan Gailey, Joyelle Johnson, Zach..."
6,2018-04-05 04:00:00,4863,Zach Sims,"Jonah Ray, Megan Gailey, Joyelle Johnson, Zach..."
7,2018-04-05 04:00:00,4863,Jono Zalay,"Jonah Ray, Megan Gailey, Joyelle Johnson, Zach..."
8,2018-03-29 04:00:00,3351,Rojo Perez,"Rojo Perez, Anna Gillcrist, Tommy McNamara, Da..."
9,2018-03-29 04:00:00,3351,Anna Gillcrist,"Rojo Perez, Anna Gillcrist, Tommy McNamara, Da..."


In [348]:
carasantamaria = xml_to_df('rss_files/carasantamaria.xml')
carasantamaria

ValueError: invalid literal for int() with base 10: '00.58.48'

In [ ]:
econtalk2013 = xml_to_df_desc('rss_files/econtalk2013.xml')
econtalk2006 = xml_to_df_desc('rss_files/econtalk2006.xml')

def clean_econtalk2006(econtalk2006):
        
    date_parser('\s(EST|GMT)$', econtalk2006)
    
    econtalk2006['guests'] = econtalk2006.description.str.extract(r'([\w\s\'\.]+)',expand=True)
    
    guest_split(', ', econtalk2006)
    guest_split(' of', econtalk2006)
    guest_split(' talks', econtalk2006)
    
    guest_split_last('Nobel Laureate ', econtalk2006)
    guest_split_last('syndicated columnist ', econtalk2006)
    guest_split_last('Investigative journalist ', econtalk2006)
    guest_split_last('Author ', econtalk2006)
    guest_split_last('Novelist ', econtalk2006)
    guest_split_last('The legendary ', econtalk2006)
            
    econtalk2006 = econtalk2006[econtalk2006.guests.str.contains('This is the') == False]
    econtalk2006 = econtalk2006[econtalk2006.guests.str.contains('EconTalk host') == False]
    econtalk2006 = econtalk2006[econtalk2006.guests.str.contains('In this bonus') == False]
    
    econtalk2006 = splitDataFrameList(econtalk2006, 'guests', ' and ')

    return econtalk2006

econtalk2006 = clean_econtalk2006(econtalk2006)

econtalk2006

## podcasts to download



In [135]:
chris_hardwick = xml_to_df('rss_files/chris_hardwick.xml')

def clean_chris_hardwick(chris_hardwick):

    date_parser('\s\-0000$', chris_hardwick)
    
    chris_hardwick['guests'] = chris_hardwick['title']
    
    guest_split(' #', chris_hardwick)
    guest_split(' Returns', chris_hardwick)
    guest_split(' returns', chris_hardwick)
    guest_split(' LIVE', chris_hardwick)
    guest_split(' Live', chris_hardwick)
    guest_split('Live', chris_hardwick)
    guest_split(' of', chris_hardwick)

    guest_split_last('w/ ', chris_hardwick)
    
    ### Manual cleaning
    
    replace('Billy West.mp3', 'Billy West', chris_hardwick)
    
    chris_hardwick = chris_hardwick.replace(r'', np.nan, regex=True)
    
    chris_hardwick = chris_hardwick[pd.notnull(chris_hardwick['guests'])]
    chris_hardwick = splitDataFrameList(chris_hardwick, 'guests', ', ')
    chris_hardwick = splitDataFrameList(chris_hardwick, 'guests', ' & ')
    chris_hardwick = splitDataFrameList(chris_hardwick, 'guests', ' And ')
    chris_hardwick = splitDataFrameList(chris_hardwick, 'guests', ' and ')
    
    return chris_hardwick

chris_hardwick = clean_chris_hardwick(chris_hardwick)

chris_hardwick[100:150]

,date,duration,guests,title
100,2017-02-07 08:00:00,4327,Keanu Reeves,Keanu Reeves Returns
101,2017-02-02 19:52:57,5984,Chilly Gonzales,Chilly Gonzales
102,2017-01-26 03:26:00,5100,SF Sketchfest with Nathan Fillion,SF Sketchfest with Nathan Fillion
103,2017-01-17 08:00:00,5651,Louie Anderson,Louie Anderson
104,2017-01-11 00:03:05,6427,Mark Hamill,Mark Hamill Returns
105,2017-01-04 21:40:58,4143,Metallica,Metallica Returns
106,2016-12-27 08:00:00,4968,2016: The Year,2016: The Year of No Chill
107,2016-12-22 02:35:58,5087,Neil Finn,Neil Finn
108,2016-12-15 17:06:39,4184,Ian McShane,Ian McShane
109,2016-12-13 08:00:00,3709,She,She & Him


In [472]:
import urllib.request

jocko = xml_to_df(urllib.request.urlopen('http://jockopodcast.libsyn.com/rss'))
jocko

,title,date,duration
0,Warrior Kid Podcast 1: Ask Uncle Jake,"Fri, 13 Apr 2018 18:41:16 +0000",1201
1,120: How to Maintain Improvement Over Time. He...,"Wed, 11 Apr 2018 06:31:53 +0000",6260
2,"Human Nature, Leadership, and Atrocities w/ Da...","Wed, 04 Apr 2018 06:26:45 +0000",9177
3,119: A Detailed Guide How to Live Life The Gen...,"Wed, 28 Mar 2018 06:11:38 +0000",10811
4,118: Always Find A Mission. And Win. With Dan ...,"Wed, 21 Mar 2018 06:13:45 +0000",6783
5,117: What to Do When You Have Bad Luck. Tactfu...,"Wed, 14 Mar 2018 06:46:30 +0000",7784
6,"116: ""I Can't"" VS. ""I Won't"". And What's Mor...","Wed, 07 Mar 2018 07:43:43 +0000",10047
7,"115: Into the Fire, and Beyond the Call of Dut...","Wed, 28 Feb 2018 06:53:16 +0000",13336
8,114: How to Lead and Win with Leif Babin.,"Wed, 21 Feb 2018 07:45:57 +0000",10908
9,113: Into the Mind of a Working K9 Warrior Wit...,"Wed, 14 Feb 2018 07:27:43 +0000",11192
